In [ ]:

from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import config
import re
import plotly.express as px


In [ ]:
# Répartition géographique des déclarants
geolocator = Nominatim(user_agent="my_user_agent")

taxes = pd.read_csv(config.EXTRACTION_FILE)
taxes = taxes.dropna(subset=['adresse','revenu_fiscal_de_ref'])

taxes['postal_code'] = taxes['adresse'].apply(
    lambda x:re.search(r'[0-9]{5}',x).group(0)[:2] if x != "" else "")

taxes['localisation'] = taxes['postal_code'].apply(
    lambda x:(geolocator.geocode(x+' France').latitude,
    geolocator.geocode(x+' France').longitude) if geolocator.geocode(x+' France') else ())


In [ ]:
count_per_area = taxes.groupby(['postal_code','localisation']).agg('count').reset_index()
count_per_area

In [ ]:
lat=np.array([tx[0] for tx in count_per_area['localisation']])
lon=np.array([tx[1] for tx in count_per_area['localisation']])

fig = px.scatter_geo(count_per_area,lat=lat,lon=lon, hover_name="postal_code",color='postal_code',size='nom')
fig.update_layout(title = 'World map', title_x=0.5)
#fig.show()

In [ ]:
# Revenus moyens par département
taxes['revenu_fiscal_de_ref'].astype('int64')
revenus_par_zone = taxes.groupby(['postal_code','revenu_fiscal_de_ref']).agg('mean').reset_index()
revenus_par_zone

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=revenus_par_zone['postal_code'],y=revenus_par_zone['revenu_fiscal_de_ref'].head(5))
plt.xticks(rotation=45)
plt.title("Revenus moyens par département")
plt.show()